# DecisionTree
## I try to use decisiontree to solve this problem.

In [1]:
import h5py
import numpy as np
import math
import time
#load data
with h5py.File('./data/train/images_training.h5', 'r') as H:
    data_train = np.copy(H['datatrain'])
with h5py.File('./data/train/labels_training.h5', 'r') as H:
    label_train = np.copy(H['labeltrain'])
with h5py.File('./data/test/images_testing.h5', 'r') as H:
    data_test = np.copy(H['datatest'])
with h5py.File('./data/test/labels_testing_2000.h5', 'r') as H:
    label_test = np.copy(H['labeltest'])


In [2]:
import math
import copy
#private variable depth is not necessary
#I hope use this variable can make the code run faster
#decision tree do not need to be binary tree, but it is the easist tree data structure, I suppose
class node:
    value = None
    left_child = None
    right_child = None
    depth = 0

    def __init__(self, value=None, depth=None):
        self.value = value
        self.depth = depth

#I find that multi-classifier is much more difficult than the ordinary one.
#So I choose to build 10 decision tree
#each tree only focus on figuring out probability that each data belongs to specific label
class DecisionTree:
    num_tree = 10
    root = [node(depth=0) for i in range(num_tree)]
    threshold = 0.2
    acce_resu = 0.8
    min_info_gain = 0.01
    max_depth = 16
    data_train = np.array([])
    label_train = np.array([])
    scale = 1.2
    
    def __init__(self, num_tree=10, threshold=0.2, acce_resu=0.8, min_info_gain=0.01, max_depth=16):
        num_tree = num_tree
        root = [node(depth=0) for i in range(num_tree)]
        threshold = threshold
        acce_resu = acce_resu
        min_info_gain = min_info_gain
        max_depth = max_depth

    def buildtree(self, point, row, column, label_num):
        max_info_gain = 0
        temp = [0, 0, [[], []], -1]
        row_len = len(row)
        new_col = copy.deepcopy(column)
        mini_set_num = 0.1*row_len
        #scan each pixel in order to find out whether the pixel can effectively divide the data
        for i in column:
            #count the numbers in order to calculate the information gain
            count = [0, 0]
            new_row = [[], []]
            for j in row:
                if self.data_train[j, i] > self.threshold:
                    new_row[1].append(j)
                    if label_train[j] == label_num:
                        count[1] = count[1]+1
                else:
                    new_row[0].append(j)
                    if label_train[j] == label_num:
                        count[0] = count[0]+1
            new_row0_len = len(new_row[0])
            new_row1_len = len(new_row[1])
            #if the whole data set cannot be divided into two parts, 
            #the classification is useless 
            if new_row1_len < mini_set_num or new_row0_len < mini_set_num:
                new_col.remove(i)
                continue
            else:
                p1 = count[0]/new_row0_len
                p3 = count[1]/new_row1_len
                #If the data is divided perfectly
                if p1 > self.acce_resu and p3 > self.acce_resu:
                    point.value = i
                    point.left_child = node(value=float(p1), depth=point.depth+1)
                    point.right_child = node(value=float(p3), depth=point.depth+1)
                    print('best')
                    return
                #If one part of the data is divied perfectly
                elif count[0] == 0 or p1 > self.acce_resu:
                    point.value = i
                    point.left_child = node(value=float(p1), depth=point.depth+1)
                    if point.depth == self.max_depth:
                        point.right_child = node(value=float(p3), depth=point.depth+1)
                    else:
                        point.right_child = node(depth=point.depth+1)
                        new_col.remove(i)
                        self.buildtree(point.right_child, new_row[1], new_col, label_num)
                    return
                #If another part of the data is divieds perfectly
                elif count[1] == 0 or p3 > self.acce_resu:
                    point.value = i
                    point.right_child = node(value=float(p3), depth=point.depth+1)
                    if point.depth == self.max_depth:
                        point.left_child = node(value=float(p1), depth=point.depth+1)
                    else:
                        point.left_child = node(depth=point.depth+1)
                        new_col.remove(i)
                        self.buildtree(point.left_child, new_row[0], new_col, label_num)
                    return
                else:
                    #if do not have the perfect attribute
                    #then calculate the information gain
                    #find the maximum and record some feature
                    p2 = 1-p1
                    p4 = 1-p3
                    old_p1 = (count[0]+count[1])/row_len
                    old_p2 = 1-old_p1
                    info_gain = new_row0_len/row_len*(p1*math.log(p1)+p2*math.log(p2))+new_row1_len/row_len*(p3*math.log(p3)+p4*math.log(p4))-old_p1*math.log(old_p1)-old_p2*math.log(old_p2)
                    if info_gain > max_info_gain:
                        max_info_gain = info_gain
                        temp = [p1, p3, new_row, i]
        
        point.value = temp[3]
        if point.depth == self.max_depth:
                    point.left_child = node(value=float(temp[0]), depth=point.depth+1)
                    point.right_child = node(value=float(temp[1]), depth=point.depth+1)
                    return

        if max_info_gain > self.min_info_gain:
            point.left_child = node(depth=point.depth+1)
            point.right_child = node(depth=point.depth+1)
            new_col.remove(temp[3])
            self.buildtree(point.left_child, temp[2][0], new_col, label_num)
            self.buildtree(point.right_child, temp[2][1], new_col, label_num)
        else:
            point.left_child = node(value=float(temp[0]), depth=point.depth+1)
            point.right_child = node(value=float(temp[1]),depth=point.depth+1)

    #there are 10 decision tree which need to build
    #use the whole data to build each tree takes too much time, I suppose
    #so randomly divide the data into 10 groups
    #and then build each trees
    def fit(self, data_train, label_train):
        self.data_train =data_train
        self.label_train = label_train
        len_row0 = self.data_train.shape[0]
        column0 = list(range(self.data_train.shape[1]))
        temp = int(len_row0/self.num_tree*self.scale)
        for i in range(self.num_tree):
            print('start to build '+str(i)+' decisiontree')
            self.buildtree(self.root[i], np.random.randint(low=0, high=len_row0, size=temp), column0, i)
    
    #the value of each node should be either the index of pixel(int) or the result probability(float) for the leaf nodes
    #as mentioned above, I build 10 decision tree and each decision tree only focus on the probability of one piece of data belonging to specific label
    #so in the 'predict' function, each piece of data must be input into the whole 10 decision trees and see which decision tree get the maximum probability
    #this piece of data is prediected to the label(the same as the index of the decision tree)
    def predict(self, data_test):
        prediction = []
        for i in data_test:
            max_poss = 0
            pos_label = -1
            for index, point in enumerate(self.root):
                while 1:
                    temp = isinstance(point.value, float)
                    if temp:
                        if max_poss < point.value:
                            max_poss = point.value
                            pos_label = index
                        elif max_poss == point.value and max_poss != 0 :
                            print('can not classify well')
                        break
                    elif isinstance(point.value, int):
                        if i[point.value] > self.threshold:
                            point = point.right_child
                            continue
                        else:
                            point = point.left_child
                            continue
            prediction.append(pos_label)
        return np.array(prediction)

In [6]:
start = time.time()
deci_tree = DecisionTree()
deci_tree.fit(data_train, label_train)
output = deci_tree.predict(data_test[:2000])
count = 0
for i in range(2000):
    if output[i] == label_test[i]:
        count = count+1
print(count/2000)
end = time.time()
print(end-start)

start to build 0 decisiontree
start to build 1 decisiontree
start to build 2 decisiontree
start to build 3 decisiontree
start to build 4 decisiontree
start to build 5 decisiontree
start to build 6 decisiontree
start to build 7 decisiontree
start to build 8 decisiontree
start to build 9 decisiontree
can not classify well
0.7055
166.67553305625916


In [4]:
len(label_test)

2000